## FFT (Fast Fourier Transform)

Szybkie przekształcenie jest algorytmem pozwalającym na redukcję złozoności obliczania dyskrentego przekształcenia Fouriera o jeden rząd, tj. ze złożoności $\mathcal{O}(N^2)$ do złożoności rzędu $\mathcal{O}(N\log_2N)$. Budowa szybkiego przekształcenia (ang. *Fast Fourier Transform* (FFT)) jest możliwa dzięki pewnym własnościom symetrii funkcji bazowych (macierzy) dyskretnego przekształcenia Fouriera i jest realizowana w myśl strategii "dziel i rządź". Wyprowadzenie FFT zamieszczono w dodatkowych materiałach. Poniżej prezentujemy wyłącznie szkielet algorytmu opartego o pętle. Nie jest algorytm rekurencyjny, chociaż rekurencja jest naturalnym narzędziem realizacji algorytmów budowanych w myśl wspomnianej strategii.

In [1]:
import numpy as np
import math

Elementy wektora podawane na wejście przekształcenia FFT muszą być przemieszane zgodnie z kolejnością *bit-reverse*. Poniższa funkcja realizuje wspomnianą permutację.

In [2]:
def bit_reverse(x,N,n):

	for i in range(0,N):
		j=0
		for k in range(0,n):
			j<<=1
			j+=(i>>k)&1
		print('i:',i,'-> j:',j)
		if (j>i):
			t=x[i]
			x[i]=x[j]
			x[j]=t
	return x

Bardzo proszę zastanowić się nad złożonością (w sensie rzędu) tej procedury. W dalszej kolejności prezentujemy szablon algorytmu FFT. Zmienna *nb* jest odpowiedzialna za liczbę bloków motylków. Ta liczba na początku wynosi $N/2$ i jest sukcesywnie zmniejszana dwukrotnie na każdym etapie działania algorytmu (tj. dla $i=0,1,...,\log_2N-1$). Druga zmianna *ne* reprezentuje liczbę motylków w każdym bloku. Na początku ta liczba wynosi jeden i jest sukcesywnie zwiększana dwukrotnie na każdym etapie. Do wyznaczania punktów przyłożenia operacji motylkowych (odpowiedzilanych za operacje dodawanie/odejmowanie) wyznaczamy według wzorów:

$i1=2*k*ne+l$,

$i2=i1+ne$,

gdzie $k=0,1,...,nb-1$ i $l=0,1,...,ne-1$.

In [3]:
def fft_template(x,N,n):

	X=np.zeros(N,complex)
	nb=N//2
	ne=1
	# motylki
	for i in range(0,n):
		print('------------------- i:',i)
		for k in range(0,nb):
			print('### -> nb:',k)
			for l in range(0,ne):
				i1=2*k*ne+l
				i2=i1+ne
				print('(',i1,',',i2,')')
		nb//=2
		ne*=2
	print('-------------------')
	return X

Poniżej zamieszczamy kod mający za zadanie przetestować działania procedury realizującej permutację, jak również samego szablonu algorytmu FFT.

In [4]:
N=16
n=int(math.ceil(math.log(N)/math.log(2)))
print('N:',N,'n:',n)
x=np.zeros(N,float)
x=np.arange(0,N)
x=bit_reverse(x,N,n)
print(x)
X=fft_template(x,N,n)

N: 16 n: 4
i: 0 -> j: 0
i: 1 -> j: 8
i: 2 -> j: 4
i: 3 -> j: 12
i: 4 -> j: 2
i: 5 -> j: 10
i: 6 -> j: 6
i: 7 -> j: 14
i: 8 -> j: 1
i: 9 -> j: 9
i: 10 -> j: 5
i: 11 -> j: 13
i: 12 -> j: 3
i: 13 -> j: 11
i: 14 -> j: 7
i: 15 -> j: 15
[ 0  8  4 12  2 10  6 14  1  9  5 13  3 11  7 15]
------------------- i: 0
### -> nb: 0
( 0 , 1 )
### -> nb: 1
( 2 , 3 )
### -> nb: 2
( 4 , 5 )
### -> nb: 3
( 6 , 7 )
### -> nb: 4
( 8 , 9 )
### -> nb: 5
( 10 , 11 )
### -> nb: 6
( 12 , 13 )
### -> nb: 7
( 14 , 15 )
------------------- i: 1
### -> nb: 0
( 0 , 2 )
( 1 , 3 )
### -> nb: 1
( 4 , 6 )
( 5 , 7 )
### -> nb: 2
( 8 , 10 )
( 9 , 11 )
### -> nb: 3
( 12 , 14 )
( 13 , 15 )
------------------- i: 2
### -> nb: 0
( 0 , 4 )
( 1 , 5 )
( 2 , 6 )
( 3 , 7 )
### -> nb: 1
( 8 , 12 )
( 9 , 13 )
( 10 , 14 )
( 11 , 15 )
------------------- i: 3
### -> nb: 0
( 0 , 8 )
( 1 , 9 )
( 2 , 10 )
( 3 , 11 )
( 4 , 12 )
( 5 , 13 )
( 6 , 14 )
( 7 , 15 )
-------------------


Ponieważ warto jest porównać wyniki uzyskane za pomocą DFT i naszej implementacji FFT podajemy poniżej implementację DFT w oparciu o proste biblioteki. Takie porównanie powinno dawać zgodne wyniki z dokładnością do pewnego (odległe, około piątego, szóstego) miejsca po przecinku. Tym samym nie należy porównywać wartości literalnie. Procedura na wejściu przyjmuje dwie tablice przedstawiające odpowiednio części rzeczywiste (xr) i urojone (xi) danych wejściowych. Na wyjściu oczekujemy również dwóch tablic reprezentujących części rzeczywiste (Xr) i urojone (Xi) współczynników Fouriera.

In [21]:
def DFT(xr,xi):
    
    N=len(xr)
    Xr=np.zeros(N,float)
    Xi=np.zeros(N,float)
    for k in range(0,N):
        for n in range(0,N):
            a=2*math.pi/N*k*n
            c=math.cos(a)
            s=math.sin(a)
            Xr[k]=Xr[k]+(c*xr[n]+s*xi[n])
            Xi[k]=Xi[k]+(c*xi[n]-s*xr[n])
    return (Xr,Xi)

def IDFT(Xr,Xi):
    
    N=len(Xr)
    xr=np.zeros(N,float)
    xi=np.zeros(N,float)
    for n in range(0,N):
        for k in range(0,N):
            a=-2*math.pi/N*k*n
            c=math.cos(a)
            s=math.sin(a)
            xr[n]=xr[n]+(c*Xr[k]+s*Xi[k])
            xi[n]=xi[n]+(c*Xi[k]-s*Xr[k])
    return (xr/N,xi/N)

# przyklad użycia
xr=np.array([1,2,3,-4],dtype=float)
xi=np.array([1,-1,-1,1],dtype=float)
(Xr,Xi)=DFT(xr,xi)
(zr,zi)=IDFT(Xr,Xi)
print(xr,'=',zr)
print(xi,'=',zi)

[ 1.  2.  3. -4.] = [ 1.  2.  3. -4.]
[ 1. -1. -1.  1.] = [ 1. -1. -1.  1.]


---